In [1]:
import torch
import numpy as np
from torch import nn
import random
import torch.nn.functional as F
import collections
from torch.optim.lr_scheduler import StepLR
# from CommunicationP3DX import CommunicationP3DX
# from Agent import AgentClass
# from std_srvs.srv import Empty
# import rospy
import time
import shelve
import os
from datetime import datetime
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def get_complete_hist_from_shelve(workspace_folder_path, agg_method, n_sectors):
    return shelve.open(workspace_folder_path+'/wsh_{metodo}{n_setores}completo.out'.format(
        metodo = agg_method,n_setores = n_sectors
    ))['hist_dict']

def get_rewards_df(hist_dict,method):
    rewards = []
    episodes = []
    for key in hist_dict['rewards'].keys():
        episodes.append(key)
        rewards.append(hist_dict['rewards_eval'][key][0])
    return pd.DataFrame({'Episodes':episodes, 'Rewards'+method:rewards})

def get_distances_df(hist_dict, method):
    distances = []
    episodes = []
    for key in hist_dict['rewards'].keys():
        episodes.append(key)
        distances.append(hist_dict['rewards_eval'][key][1])
    return pd.DataFrame({'Episodes':episodes, 'Distances'+method:distances})

# 0 for crash, 1 for goal 2 for time
def get_success_df(hist_dict,method):
    success = []
    episodes = []
    for key in hist_dict['epresult'].keys():
        episodes.append(key)
        success.append(hist_dict['epresult'][key][1])
    df = pd.DataFrame({'Episodes':episodes, 'Success'+method:success})
    df['Success'+method] = np.where(df['Success'+method]==1,1,0)
    return df

def get_collision_df(hist_dict,method):
    collision = []
    episodes = []
    for key in hist_dict['epresult'].keys():
        episodes.append(key)
        collision.append(int(np.where(hist_dict['epresult'][key][1]==0,1,0)))
    df = pd.DataFrame({'Episodes':episodes, 'Collision'+method:collision})
    return df

training_list = [
    ['mean','4']
    ,['mean','5']
    ,['mean','6']
    ,['mean','10']
    ,['min','4']
    ,['min','5']
    ,['min','6']
    ,['min','10']
    ]

# get hists by method
workspace_folder_path = '/media/xnd/7A309A87309A49D1/sia_23/25k/'
#  'C:/Users/alexa/Desktop/sia_DRL_2023/25k/'


In [2]:
hist_dict = get_complete_hist_from_shelve(workspace_folder_path, agg_method='mean', n_sectors='6')
# hist_dict['pos_eval'].keys(0)

In [33]:
def get_pos_df(hist_dict):
    pos_df = pd.DataFrame({'x':[], 'y':[], 'iteration':[], 'episode':[]})
    poskey='pos'

    for key in hist_dict[poskey].keys():
        episode = key

        x = []
        y = []
        its=[]
        for it_idx,_ in enumerate(hist_dict[poskey][episode]):
            x.append(hist_dict[poskey][episode][it_idx][0])
            y.append(hist_dict[poskey][episode][it_idx][1])
            its.append(int(it_idx))

        pos_df = pd.concat([pos_df,
                            pd.DataFrame({'x':x, 'y':y, 'iteration':its, 'episode':[int(episode) for _ in its]})])
    
    return pos_df

pos_df = get_pos_df(hist_dict)
pos_df

,x,y,iteration,episode
0,-8.861144,7.788044,0.0,1.0
1,-2.934483,3.238775,1.0,1.0
0,-3.582236,1.853754,0.0,2.0
1,-2.761467,0.938100,1.0,2.0
2,-2.176662,0.759816,2.0,2.0
...,...,...,...,...
12,-7.575229,5.076172,12.0,25000.0
13,-8.514272,5.958520,13.0,25000.0
14,-8.462346,6.785835,14.0,25000.0
15,-8.043694,7.293556,15.0,25000.0


In [70]:
def get_scan_df(hist_dict, scankey='scan'):
    
    scan_df = pd.DataFrame(columns = ['iteration','episode','scan'])
        # columns=['iteration','episode',*['scan_'+str(scan_idx) for scan_idx in range(0,727)]])
    
    for key in hist_dict[scankey].keys():

        episode = key
        its = []
        scans = []

        for it_idx,_ in enumerate(hist_dict[scankey][episode]):

            scans.append(np.asarray(hist_dict['scan'][episode][it_idx]).transpose())
            its.append(int(it_idx))

        scan_df = pd.concat([scan_df,
                            pd.DataFrame({'scan':scans,'iteration':its, 'episode':[int(episode) for _ in its]})])
        
    return scan_df

scan_df = get_scan_df(hist_dict)
scan_df

,iteration,episode,scan
0,0,1,"[2.4994232654571533, 2.4923508167266846, 2.515..."
1,1,1,"[3.5340607166290283, 3.5436127185821533, 3.553..."
0,0,2,"[inf, inf, inf, inf, inf, inf, inf, inf, inf, ..."
1,1,2,"[3.4302003383636475, 3.3355414867401123, 3.245..."
2,2,2,"[0.6349397301673889, 0.6399048566818237, 0.625..."
...,...,...,...
12,12,25000,"[0.7305604219436646, 0.733041524887085, 0.7290..."
13,13,25000,"[inf, inf, inf, inf, inf, inf, inf, inf, inf, ..."
14,14,25000,"[inf, inf, inf, inf, inf, inf, inf, inf, inf, ..."
15,15,25000,"[inf, inf, inf, inf, inf, inf, inf, inf, inf, ..."


In [ ]:
# hist_dict['pos_eval'].keys(0)
df = pd.DataFrame.from_dict(hist_dict['pos_eval'], orient='index',
                       columns=['ep_eval', 'pos'])

df

In [ ]:
# f, ax = plt.subplots(figsize=(6, 6))
# sns.kdeplot(data = pos_df,x = 'x', y = 'y', fill=True, thresh=0, cmap='Blues');

In [ ]:
# import pickle
# with open('df_episode_positions_mean_min_25k.pickle', 'wb') as handle:
#     pickle.dump(df_episode_positions, handle, protocol=pickle.HIGHEST_PROTOCOL)